### Let's try cox survival analysis here with a reduced dataset

In [ ]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#load initial cleaned data
df=pd.read_csv(r"..\data\processed\prelim_clean2.csv", index_col=[0])
df.drop(['visit_interval'],axis=1, inplace=True)
df2=pd.read_csv(r"..\data\processed\prelim_clean2.csv", index_col=[0])

In [ ]:
broad_features = pickle.load(open('../models/01final_features_broad.sav', 'rb'))
res_features = pickle.load(open('../models/01final_features_res.sav', 'rb'))

In [ ]:
broad_subset=df[df.columns.intersection(broad_features)]
subset=df[df.columns.intersection(res_features)]

In [ ]:
from lifelines import CoxPHFitter
from lifelines.utils import k_fold_cross_validation

broad_subset['visit_interval']=df2['visit_interval']
broad_subset['target']=df2['target']

cph = CoxPHFitter(penalizer=0.1)
#scores = k_fold_cross_validation(cph, broad_subset, 'visit_interval', event_col='target', k=10)
#print(scores)

# scores = k_fold_cross_validation(cph, broad_subset, duration_col='visit_interval', event_col='target', k=10, scoring_method="concordance_index")
# print(scores)

cph.fit(df=broad_subset, duration_col='visit_interval', event_col='target')
cph.print_summary()


In [ ]:
subset['visit_interval']=df2['visit_interval']
subset['target']=df2['target']

cph = CoxPHFitter(penalizer=0.01)
scores = k_fold_cross_validation(cph, subset, duration_col='visit_interval', event_col='target', k=10, scoring_method="concordance_index")
print(scores)

cph.fit(df=broad_subset, duration_col='visit_interval', event_col='target')
#cph.print_summary()

In [ ]:
# from lifelines.utils.sklearn_adapter import sklearn_adapter

# from lifelines import CoxPHFitter
# from lifelines.datasets import load_rossi

# X = load_rossi().drop('week', axis=1) # keep as a dataframe
# Y = load_rossi().pop('week')

# CoxRegression = sklearn_adapter(CoxPHFitter, event_col='arrest')
# # CoxRegression is a class like the `LinearRegression` class or `SVC` class in scikit-learn

# sk_cph = CoxRegression(penalizer=1e-5)
# sk_cph.fit(X, Y)
# print(sk_cph)

# """
# SkLearnCoxPHFitter(alpha=0.05, penalizer=1e-5, strata=None, tie_method='Efron')
# """

# sk_cph.predict(X)
# sk_cph.score(X, Y)

## Cox with skurv (just to test how functions work in their data)

In [ ]:
import numpy
from sksurv.datasets import load_gbsg2
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import integrated_brier_score
from sksurv.preprocessing import OneHotEncoder

In [ ]:
X, y = load_gbsg2()
X.loc[:, "tgrade"] = X.loc[:, "tgrade"].map(len).astype(int)
Xt = OneHotEncoder().fit_transform(X)

In [ ]:
est = CoxPHSurvivalAnalysis(ties="efron").fit(Xt, y)

In [ ]:
survs = est.predict_survival_function(Xt)
times = numpy.arange(365, 1826)
preds = numpy.asarray([[fn(t) for t in times] for fn in survs])

In [ ]:
survs[0]

In [ ]:
score = integrated_brier_score(y, y, preds, times)
print(score)

# survival forest with sksurv

In [ ]:
from sksurv.ensemble import RandomSurvivalForest
from sksurv.util import Surv

In [ ]:
y=Surv.from_dataframe('target', 'visit_interval', df2)
X_train, X_test, y_train, y_test = train_test_split(subset, y, test_size=0.35, random_state=1)


In [ ]:
rsf = RandomSurvivalForest()
rsf.fit(X_train, y_train)

### Assessing model

In [ ]:
rsf.score(X_test, y_test)

In [ ]:
surv = rsf.predict_survival_function(X_test)

surv_p=pd.Series(rsf.predict(X_test))

In [ ]:
surv_p

In [ ]:
survs = rsf.predict_survival_function(X_test, return_array=True)
times=rsf.event_times_
haz=rsf.predict_cumulative_hazard_function(X_test, return_array=True)

In [ ]:
times.shape

In [ ]:
time=np.arange(6, 400+6) #6.0; 2504

In [ ]:
from sksurv.metrics import integrated_brier_score, brier_score
integrated_brier_score(y_train, y_test, survs, time)# not implemented for trees in this library =( super hacked, but worked!

In [ ]:
from sksurv.metrics import concordance_index_ipcw,cumulative_dynamic_auc
concordance_index_ipcw(y_train, y_test, survs[:,0])

In [ ]:
t=[]
s=[]

for i in range(399):
    time, score = brier_score(y_train, y_test, survs[:,i], times[i])
    t.append(time)
    s.append(score)

plt.plot(t, s)
plt.axhline(0.194, linestyle="--")
plt.xlabel("days from enrollment")
plt.ylabel("Prediction error")
plt.grid(True)


In [ ]:
ix = [i[0] for i in y_test]
sum(ix)
len(ix)

In [ ]:
nh=np.mean(haz[ix,:],axis=0)
yh=np.mean(haz[~np.array(ix),:],axis=0)

times=rsf.event_times_
plt.plot(times, nh)
plt.plot(times, yh)

plt.xlabel("days from enrollment")
plt.ylabel("Hazard rate")
plt.legend(['AD','MCI'])

plt.grid(True)


In [ ]:
pickle.dump(rsf, open('../models/surv_model.sav', 'wb'))

In [ ]:
for i, s in enumerate(surv):
    plt.step(rsf.event_times_, s, where="post", label=str(i))
plt.ylabel("Survival probability")
plt.xlabel("Time in days")
plt.grid(True)
plt.legend()

In [ ]:
plt.step(rsf.event_times_, surv[0], where="post")